# Writing Continuous Applications with Structured Streaming Python APIs in Apache Spark

Tutorial for Spark Streaming and ML

At first glance, building a distributed streaming engine might seem as simple as launching a set of servers and pushing data between them. Unfortunately, distributed stream processing runs into multiple complications that don’t affect simpler computations like batch jobs. Fortunately, PySpark 2.4 and Databricks makes this simple!

This notebook shows how one can train a model using Apache Spark and MLlib then deploy that model using Spark's structured streaming for making predictions as a continunous application.

This example will use a credit card fraud use case to demonstrate how MLlib models and structured streaming can be combined, to constitutue a continunous application. In our hypothetical use case, we have some historical data of credit card transactions, some of which have been identified as fraud. We want to train a model using this historical data that can flag potentially fraudulent transactions coming in as a live stream. We then want to deploy that model as part of a data pipeline which will work with a stream of transaction data to identify potential fraud hotspots in a continunous manner.

-sandbox
<div style="line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/04/PySparkStructuredStreaming-1.jpg" alt="Structrured Streaming" width="50%" style=>
</div>

The dataset has been collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

Please cite: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

A copy of this data and its licence are available at https://s3-us-west-2.amazonaws.com/ml-team-public-read/credit-card-fraud.zip

This dataset has 3 columns we'll be using.

**pcaVector:** The PCA transformation of raw transaction data. The main idea of principal component analysis (PCA) is to reduce the dimensionality of a data set consisting of many variables correlated with each other. Put simply, it is a method of summarizing data.

**amountRange:** This column is a value between 0 and 7 and tells us the approximate amount of a transaction. The values correspond to 0-1, 1-5, 5-10, 10-20, 20-50, 50-100, 100-200, and 200+ in dollars.

**label:** 0 or 1, whether a transaction was fraudulent.

We want to build a model which will predict the label using the pcaVector and amountRange data. We'll do this by using a ML pipeline with 3 stages:
* 1) A **OneHotEncoder** to build a vector from our _amountRange_ column. It is a process by which categorical variables are converted into a vector form that could be provided to ML algorithms to do a better job in prediction.
* 2) A **Vector assembler** to merge our _pcaVector_ & _amountRange_ vector into our features vector. It is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees 
* 3) A **GBTClassifier** to serve as our Estimator. It's a learning algorithm for classification. It supports binary labels, as well as both continuous and categorical features.

## Setup input and output files

In [6]:
input_data = "/databricks-datasets/credit-card-fraud/data"
output_test_parquet_data = "/tmp/pydata/credit-card-frauld-test-data"

In [7]:
#
# Lets take a look at the schema of the historical dataset we'll be working with today
#
data = spark.read.parquet(input_data)
display(data)

time,amountRange,label,pcaVector
52972,2,0,"List(1, 28, List(), List(-0.775460885847953, 0.709595287942808, 1.61012185737375, 1.23179224889532, 0.316177505485586, 0.11441004444483, 0.410964424898908, 0.0482463120477359, 0.0160424613851936, 0.590293137181086, 1.06963744560403, -0.0970015458011077, -2.05395393083259, 0.296433513650427, -0.242125977744976, -1.10398301441835, 0.438081447824287, -0.0900982397359751, 0.972524799861282, -0.0095254201744326, -0.0313171825598146, 0.289930092783562, -0.215881463615024, 0.196378667308146, -0.267693005539635, -0.298693369594354, 0.0111946468450594, -0.0233199829808697))"
41768,6,0,"List(1, 28, List(), List(0.873553941009571, -1.37751021440628, 1.07203129318284, 0.898335056199892, -1.37767734218766, 1.1341181119011, -1.04950509839334, 0.303593246528786, 0.391174678156379, 0.30645706157889, -1.74332056201875, 0.496823246907654, 0.582594013119008, -1.04298111816545, -1.09304763035534, -2.15804820205749, 0.552883029565036, 0.941659626723935, -0.878165967036597, -0.188112036138337, -0.206446493695106, -0.0208600286360115, -0.294525977933785, -0.419986418387049, 0.530461547583973, -0.0953143171911623, 0.102469833198419, 0.0613929441495575))"
40769,7,0,"List(1, 28, List(), List(0.890896976094619, -0.528186780186241, -0.678653636768693, 0.168700533941972, -0.141127297738687, -1.03734480197013, 0.786720278659216, -0.463098544159764, -0.231256401727913, -0.210526809038417, -0.579607229088567, 0.320748654905059, 0.739512636179789, 0.385612185997922, 0.760072091416543, 0.0562860882084142, -0.365932329362014, -0.553207062295103, 0.225507413957292, 0.469011543311791, 0.0724908019085573, -0.283582883371654, -0.367601705300131, -0.00374237527132833, 0.548135925665169, 1.07383485281209, -0.156020333403545, 0.0292101280471998))"
40682,3,0,"List(1, 28, List(), List(-0.572954136732452, 0.458245881426411, 0.239597590908972, -1.56195349253073, 2.68806291830668, 3.5295007407309, 0.386101316501381, 0.773473570610461, 0.0616701240466451, -0.425875341967141, -0.594523365105178, -0.149708184135855, -0.36862982219484, -0.285347030554428, -0.600321621523469, 0.106861235199372, -0.793362789636541, -0.20800446692985, 0.0823295987619518, 0.22946073939933, -0.290700364845232, -0.627816849509753, -0.206939602558071, 1.02190677932925, 0.191260521903609, 0.219702773949549, 0.122007212047274, -0.0629967134534869))"
50032,7,0,"List(1, 28, List(), List(-2.05305889575943, 0.504530324242776, -0.111155667869986, -1.04073848891865, -2.68458996781652, 0.0484387282726955, 2.15221951164238, 0.497990320120156, -0.802817358608386, -1.26277208511232, 1.10103363698183, 0.617549672582074, -0.478061657274302, 0.968828948292094, -0.337550992626504, 1.02051282591686, -0.476523815268263, 0.0486324920932011, -0.389469468851195, -0.142086381219838, -0.0511834359345369, -0.370120210625707, 0.583369828558284, 0.556770951700458, -0.250298033074819, 0.631668074606364, 0.103109972579581, -0.186395682097962))"
53637,7,0,"List(1, 28, List(), List(0.593549262043115, -1.0453650732411, -0.610745896437447, 0.277111810627197, -0.583285921921287, -0.995132318101401, 0.732746548626124, -0.283726190338305, -0.289380804099332, -0.0467347939072331, 1.28447592477553, 0.303680861144012, -1.05582055096572, 0.964711022411289, 0.217875962525365, 0.199900565624148, -0.338876909890541, 0.00498898793577377, 0.302888768406764, 0.588759417414082, 0.206397605941981, -0.273923925471225, -0.398510675654682, 0.401092217682281, 0.35465218080797, 1.01903912167697, -0.191281030233503, 0.0402095546132128))"
39160,6,0,"List(1, 28, List(), List(-0.923235255508135, -0.445387319467476, 1.42286311943015, -1.40924129397362, 0.498512239591899, 0.746435856726508, 1.08478835512332, -0.3917344097041, -0.827226697036681, 0.291464719918271, -1.75828968368404, -0.593144215020114, 1.0819478139191, -0.891649065339908, 0.172534238233272, -0.52787562413733, -1.5225349679792, 1.41081928493877, -2.7834715943276, -0.49615003392596, -0.299173667118795, -0.0188754189035958, 0.0207146225103749, -1.381

In [8]:
data.count()

Out[3]: 284807

We using PySpark so import the appropriate classes

In [10]:
from pyspark.ml.feature import OneHotEncoderEstimator, VectorAssembler, VectorSizeHint
from pyspark.ml.classification import GBTClassifier

from pyspark.sql.types import *
from pyspark.sql.functions import count, rand, collect_list, explode, struct, count

The way we do this will be very familiar to anyone who has used MLlib, but because we intend to use this model in a streaming context, there a few things we should be aware of.

First, you may notice that we used a `OneHotEncoderEstimator`, which is new in Spark 2.3, and not a OneHotEncoder, which has now been deprecated. This new estimator fixes several issues related of the `OneHotEncoder` and will also allow you to do one hot encoding on streaming dataframes.

And the second thing to be aware of when using MLlib with structured streaming is that `VectorAssembler` has some limitations in a streaming context. Specifically, `VectorAssembler` can only work on Vector columns of known size. To address this issue we can explicitly specify the size of the pcaVector column so that we'll be be able to use our pipeline with structured streaming. To do this we'll use the `VectorSizeHint` transformer.

In [12]:
oneHot = OneHotEncoderEstimator(inputCols=["amountRange"], outputCols=["amountVect"])

vectorAssembler = VectorAssembler(inputCols=["amountVect", "pcaVector"], outputCol="features")

estimator = GBTClassifier(labelCol="label", featuresCol="features")

In [13]:
from pyspark.ml.feature import VectorSizeHint

vectorSizeHint = VectorSizeHint(inputCol="pcaVector", size=28)

### Now we're ready to build a our ML Pipeline and fit it.

In [15]:
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

pipeline = Pipeline(stages=[oneHot, vectorSizeHint, vectorAssembler, estimator])
#
# let's split the data into testing and training datasets. 
# We will shave the test dataset for later
#
#
train = data.filter(col("time") % 10 < 8)
test = data.filter(col("time") % 10 >= 8)
#
# save our data into partitions so we can read them as files
#
(test.repartition(20).write
  .mode("overwrite")
  .parquet(output_test_parquet_data))

In [16]:
%sh
du --human-readable --max-depth=1 --exclude='/dbfs' /

16K	/mnt
9.3M	/bin
2.5G	/usr
4.0K	/dev
120K	/tmp
du: cannot read directory '/proc/tty/driver': Permission denied
du: cannot read directory '/proc/1247/task/1247/fd': Permission denied
du: cannot read directory '/proc/1247/task/1247/fdinfo': Permission denied
du: cannot read directory '/proc/1247/task/1247/ns': Permission denied
du: cannot read directory '/proc/1247/task/1459/fd': Permission denied
du: cannot read directory '/proc/1247/task/1459/fdinfo': Permission denied
du: cannot read directory '/proc/1247/task/1459/ns': Permission denied
du: cannot read directory '/proc/1247/task/1460/fd': Permission denied
du: cannot read directory '/proc/1247/task/1460/fdinfo': Permission denied
du: cannot read directory '/proc/1247/task/1460/ns': Permission denied
du: cannot read directory '/proc/1247/fd': Permission denied
du: cannot read directory '/proc/1247/map_files': Permission denied
du: cannot read directory '/proc/1247/fdinfo': Permission denied
du: cannot read directory '/proc/1247/ns': Permission denied
du: cannot read directory '/proc/1250/task/1250/fd': Permission denied
du: cannot read directory '/proc/1250/task/1250/fdinfo': Permission denied
du: cannot read directory '/proc/1250/task/1250/ns': Permission denied
du: cannot read directory '/proc/1250/fd': Permission denied
du: cannot read directory '/proc/1250/map_files': Permission denied
du: cannot read directory '/proc/1250/fdinfo': Permission denied
du: cannot read directory '/proc/1250/ns': Permission denied
du: cannot read directory '/proc/2005/task/2005/fd': Permission denied
du: cannot read directory '/proc/2005/task/2005/fdinfo': Permission denied
du: cannot read directory '/proc/2005/task/2005/ns': Permission denied
du: cannot read directory '/proc/2005/task/2015/fd': Permission denied
du: cannot read directory '/proc/2005/task/2015/fdinfo': Permission denied
du: cannot read directory '/proc/2005/task/2015/ns': Permission denied
du: cannot read directory '/proc/2005/fd': Permission denied
du: cannot read directory '/proc/2005/map_files': Permission denied
du: cannot read directory '/proc/2005/fdinfo': Permission denied
du: cannot read directory '/proc/2005/ns': Permission denied
du: cannot read directory '/proc/2017/task/2017/fd': Permission denied
du: cannot read directory '/proc/2017/task/2017/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2017/ns': Permission denied
du: cannot read directory '/proc/2017/task/2018/fd': Permission denied
du: cannot read directory '/proc/2017/task/2018/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2018/ns': Permission denied
du: cannot read directory '/proc/2017/task/2019/fd': Permission denied
du: cannot read directory '/proc/2017/task/2019/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2019/ns': Permission denied
du: cannot read directory '/proc/2017/task/2020/fd': Permission denied
du: cannot read directory '/proc/2017/task/2020/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2020/ns': Permission denied
du: cannot read directory '/proc/2017/task/2021/fd': Permission denied
du: cannot read directory '/proc/2017/task/2021/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2021/ns': Permission denied
du: cannot read directory '/proc/2017/task/2022/fd': Permission denied
du: cannot read directory '/proc/2017/task/2022/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2022/ns': Permission denied
du: cannot read directory '/proc/2017/task/2023/fd': Permission denied
du: cannot read directory '/proc/2017/task/2023/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2023/ns': Permission denied
du: cannot read directory '/proc/2017/task/2024/fd': Permission denied
du: cannot read directory '/proc/2017/task/2024/fdinfo': Permission denied
du: cannot read directory '/proc/2017/task/2024/ns': Permission denied
du: cannot read directory '/proc/2017/task/2025/fd': Permission denied
du: cannot re

In [17]:
%sh
rm -rf /dbfs/local_disk0/tmp/*

In [18]:
train.count()

Out[13]: 227570

In [19]:
test.count()

Out[14]: 57236

## Let's fit the model with our training data

In [21]:
pipelineModel = pipeline.fit(train)

We can simulate a stream by reading our test data from a file, since we don't have a Kafka cluster availale for the demo.
But the effect is no different; you are still using PySpark APIs to read off the filesystem as you would off Kafka topics.

First, let's define the schema

In [23]:
from pyspark.sql.types import *
from pyspark.ml.linalg import VectorUDT

schema = (StructType([StructField("time", IntegerType(), True), 
                      StructField("amountRange", IntegerType(), True), 
                      StructField("label", IntegerType(), True), 
                      StructField("pcaVector", VectorUDT(), True)]))

## **Start:** 
Read files simulating as a Kafka stream using one file at a time

In [25]:
streamingData = (spark.readStream 
                 .schema(schema) 
                 .option("maxFilesPerTrigger", 1) 
                 .parquet(output_test_parquet_data)) # our test data

Transform the Streaming DataFrame using the model and use DataFrame PySpark API to make queries

In [27]:
from pyspark.sql.functions import *

stream = pipelineModel.transform(streamingData)

## Do aggregations using PySpark DataFrame APIs

1. _groupBy_("label", "preditcions")
2. _sort_("label", "predictions")

And finally _display()_ the predictions as they are scored in real-time from the stream

In [29]:
streamPredictions = (pipelineModel.transform(streamingData) #infer or score against our test data
          .groupBy("label", "prediction")
          .count()
          .sort("label", "prediction"))

In [30]:
display(streamPredictions)

label,prediction,count
0,0.0,57130
0,1.0,10
1,0.0,20
1,1.0,76


### HOME WORK CHALLENGE-1:
Can you compute the Precision and Recall?

Note: The formula to compute Precision (P) and Recall (r)

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2019/01/Screen-Shot-2019-01-02-at-9.38.02-AM.png" alt="Structrured Streaming" width="20%" style=>
</div>

In [32]:
display(stream)

time,amountRange,label,pcaVector,amountVect,features,rawPrediction,probability,prediction
128558,4,0,"List(1, 28, List(), List(2.01014615838007, -0.216784245141963, -1.51905032681858, 0.761876262008483, 0.0979774709440138, -0.369542727577924, -0.193014078916939, -0.120119092411686, 1.57771887198576, -0.770421845422447, -1.72452385470898, 0.225326823478681, -0.0469071147067369, -1.62546733480969, -1.01495835935236, -0.304347218204019, 0.81039349572153, 0.550321018435229, 0.351115783874138, -0.174150855822753, 0.132072610435969, 0.866695253831954, -0.231210136549362, -0.731057267942861, 0.518090693058046, 0.114701723294505, 0.0344216157657866, -0.0343935512429537))","List(0, 7, List(4), List(1.0))","List(1, 35, List(), List(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 2.01014615838007, -0.216784245141963, -1.51905032681858, 0.761876262008483, 0.0979774709440138, -0.369542727577924, -0.193014078916939, -0.120119092411686, 1.57771887198576, -0.770421845422447, -1.72452385470898, 0.225326823478681, -0.0469071147067369, -1.62546733480969, -1.01495835935236, -0.304347218204019, 0.81039349572153, 0.550321018435229, 0.351115783874138, -0.174150855822753, 0.132072610435969, 0.866695253831954, -0.231210136549362, -0.731057267942861, 0.518090693058046, 0.114701723294505, 0.0344216157657866, -0.0343935512429537))","List(1, 2, List(), List(1.5405578764817465, -1.5405578764817465))","List(1, 2, List(), List(0.9561070325471651, 0.043892967452834886))",0.0
151258,7,0,"List(1, 28, List(), List(1.49788391430533, -0.807272097411935, -2.76067480024433, 0.665143563513811, 0.394148615836649, -1.68425444456163, 1.48286626112381, -0.747799191023747, 0.0559565266202644, -0.0799852144070701, -0.75449211173009, 0.197112610736654, -0.340941729460045, 1.00890744271579, -0.0225586925979921, -1.05039363262304, -0.0213411255129481, -0.531584541394471, -0.0823428762054854, 0.369723859230606, 0.494458567927083, 0.912794146840522, -0.498701004960186, 0.079231231359936, 0.687531759983896, 0.0912195685168509, -0.136205751887537, -0.0293710358641785))","List(0, 7, List(), List())","List(1, 35, List(), List(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.49788391430533, -0.807272097411935, -2.76067480024433, 0.665143563513811, 0.394148615836649, -1.68425444456163, 1.48286626112381, -0.747799191023747, 0.0559565266202644, -0.0799852144070701, -0.75449211173009, 0.197112610736654, -0.340941729460045, 1.00890744271579, -0.0225586925979921, -1.05039363262304, -0.0213411255129481, -0.531584541394471, -0.0823428762054854, 0.369723859230606, 0.494458567927083, 0.912794146840522, -0.498701004960186, 0.079231231359936, 0.687531759983896, 0.0912195685168509, -0.136205751887537, -0.0293710358641785))","List(1, 2, List(), List(1.5432148567499864, -1.5432148567499864))","List(1, 2, List(), List(0.9563295004802634, 0.04367049951973656))",0.0
130479,6,0,"List(1, 28, List(), List(-0.570679531028558, 0.53923506919921, -0.0688610235351276, 0.0771127815319824, 0.821809208801503, -0.56386950055526, 1.12833500831135, -0.0812799633258997, -0.669054517688235, -0.95207022801489, 0.930154404494761, -0.182419062573762, -0.675156176265849, -0.513431435702028, 0.636620871891248, 0.0445078970997334, 0.557359566524728, 1.49060555266806, 0.932519338347761, 0.337664041283313, 0.381147291297006, 0.896064768933327, -0.172396642501135, -0.483318318847152, 0.224396485178895, 0.0783114658667568, 0.0985622762714588, 0.163281090072945))","List(0, 7, List(6), List(1.0))","List(1, 35, List(), List(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.570679531028558, 0.53923506919921, -0.0688610235351276, 0.0771127815319824, 0.821809208801503, -0.56386950055526, 1.12833500831135, -0.0812799633258997, -0.669054517688235, -0.95207022801489, 0.930154404494761, -0.182419062573762, -0.675156176265849, -0.513431435702028, 0.636620871891248, 0.0445078970997334, 0.557359566524728, 1.49060555266806, 0.932519338347761, 0.337664041283313, 0.381147291297006, 0.896064768933327, -0.172396642501135, -0.483318318847152, 0.224396485178895, 0.0783114658667568,

In [33]:
metrics=stream.agg(count(when((col('label')==1)&(col('prediction')==1), True)).alias('TP')
              , count(when((col('label')==0)&(col('prediction')==1), True)).alias('FP')
              , count(when((col('label')==1)&(col('prediction')==0), True)).alias('FN')
              , count(when((col('label')==0)&(col('prediction')==0), True)).alias('TN'))

In [34]:
display(metrics)

TP,FP,FN,TN
76,10,20,57130


In [35]:
import pyspark.sql.functions as F

stream_evaluator = metrics\
                 .withColumn("Precision", (F.col("TP") / (F.col("TP") + F.col("FP"))))\
                 .withColumn("Recall", (F.col("TP") / (F.col("TP") + F.col("FN"))))

In [36]:
display(stream_evaluator)

TP,FP,FN,TN,Precision,Recall
76,10,20,57130,0.8837209302325582,0.7916666666666666


This notebook demonstrates that MLlib Transformers, including PipelineModels, can be applied to streaming DataFrames. Except for the minor differences described in this notebook, you can work with structured streaming DataFrames the same way you would with batch DataFrames using MLlib.

That's the beauty of unified apis in Apache Spark 2.x and the ability to work with streaming DataFrames as static ones. 

Very cool!

### HOME WORK CHALLENGE-2:
Can you compute the F1 score, after computing the Precision and Recall?

**Note**: F1 Score = 2(PR/P + R)

In [39]:
withF1 = stream_evaluator\
                 .withColumn("F1", 2*(F.col("Precision") * F.col("Recall")) / (F.col("Precision") + F.col("Recall")))

In [40]:
display(withF1)

TP,FP,FN,TN,Precision,Recall,F1
76,10,20,57130,0.8837209302325582,0.7916666666666666,0.8351648351648352


In [41]:
dbutils.fs.rm(output_test_parquet_data, True)

Continuous Credit Card Fraud Detection with Structured Streaming Python APIs in Apache Spark
1. Set up input and output files
2. Build a machine learning pipeline with Onehot Encoder, Vector Assembler, and GBT Classifier with ML Pipeline APIs
3. Use the pipeline to feature engineer and train Gradient Boosting Tree Classifier on historical training data
4. Use the trained model to detect credit card fraud on the streaming test data
5. Analyze the results with PySpark Dataframe APIs and SQL to compute streaming precision, recall and F1 score, and achieve 0.88 precision, 0.79 recall and an F1 score of 0.84

1. Built a continuous credit card fraud detection application with structured streaming Python APIs in Apache Spark
2. Setup input and output files, split training and test data, and save test data into Parquet with partitions so we can read them as streaming files
3. Built a machine learning pipeline with One Hot Encoder, Vector Assembler, and GBT Classifier via pyspark.ml, used the pipeline to perform feature engineering and train Gradient Boosting Tree Classifier on historical training data, and used the trained model to predict credit card fraud on the livestreamed test data
4. Analyzed the predictions with PySpark DataFrame and SQL to compute streamed precision, recall and F1 score, and achieve 0.88 precision, 0.79 recall and an F1 score of 0.84

#####This project is for writing continuous applications with structured streaming Python APIs in Apache Spark

Here are some major steps:

#####Data preparation

Setup input and output files 

Create oneHot encoding for converting categorical variables to vectors.

Create Vector assesmbler to combine multiple columns into one single features vector, to train models like logistic regression or decision tree. VectorAssembler can only work on Vector columns of known size. To address this issue we can explicitly specify the size of the pcaVector column and use VectorSizeHint transformer, to apply our pipeline with structured streaming. 

Set up GVTClassifier to serve as our estimator.

#####Model Setup

Build ML pipeline wwith oneHot, vectorSizeHint, vectorAssembler, estimator.

Separate train and test data.

Save data into Parquet with partitions so we can read them as files.

#####Model Fitting

Fit model on training data.

Read in stream files with one file at a time, using pre-defined schema.

Transform the streaming dataframe using the model. 

#####Model Evaluation

Using prediction data and label, calculate precision, recall and F1 scores, group by 20 seconds window.